We import `netgen.gui` to open the netgen GUI for displaying the ferromagnet. 
We then define the geometry, and draw the box. 
The refinement of the mesh is controlled by `H_MAX` in `ngmesh`.

In [1]:
from netgen.csg import *
from ngsolve import *
from ngsolve.utils import (
    Grad,
)  # If I don't import these explicitly, VSCode reads them as missing.
import netgen.gui  # this opens up the netgen ui
import Magnetisation_Functions as magfunc
import Elastic_Functions as elfunc
import General_Functions as genfunc
import numpy as np
import scipy.sparse as sp
import time
T_MAX: float = 10.0  # The maximum time for the simulation
ALPHA: float = 1.0  # Dissipative constant in the LLG equation.
THETA: float = 0.75  # Should be strictly above 1/2 for unconditional stability
K: float = 0.1  # TIME STEP
KAPPA: float = 1.0  # Determines the relative strength of the elastic vs. magnetic parts.
H_MAX: float = 0.1  # Determines how fine the mesh should be.

In [2]:
def MakeGeometry():  # this makes a box, with labelled faces
    geometry = CSGeometry()
    left = Plane(Pnt(0, 0, 0), Vec(-1, 0, 0)).bc("left")
    right = Plane(Pnt(1, 1, 1), Vec(1, 0, 0)).bc("right")
    front = Plane(Pnt(0, 0, 0), Vec(0, -1, 0)).bc("front")
    back = Plane(Pnt(1, 1, 1), Vec(0, 1, 0)).bc("back")
    bot = Plane(Pnt(0, 0, 0), Vec(0, 0, -1)).bc("bot")
    top = Plane(Pnt(1, 1, 1), Vec(0, 0, 1)).bc("top")

    cube = left * right * front * back * bot * top
    geometry.Add(cube)
    # cube = OrthoBrick(Pnt(0,0,0), Pnt(1,1,1))
    geometry.Add(cube)
    return geometry


ngmesh = MakeGeometry().GenerateMesh(maxh=H_MAX)
# ngmesh.Save("cube.vol")
mesh = Mesh(ngmesh)
Draw(mesh)

In [3]:
# this helps me figure out which sides are which.
# fes_SCALAR = H1(mesh, order=1)
# myScalars = {"left":10, "right":10, "front":0, "back":0, "bot":5, "top":5}
# SCALAR_gfu = CoefficientFunction([myScalars[val] for val in mesh.GetBoundaries()])
# Draw(SCALAR_gfu, mesh, "SCALAR_gfu")

In [4]:
fes_mag = VectorH1(
    mesh, order=1
)  # the finite element space for the magnetisation m_h^i
fes_eps_m = MatrixValued(
    H1(mesh, order=1), dim=3
)  # matrix FE space on the magnetic part
fes_disp = VectorH1(
    mesh, order=1, dirichlet="bot"
)  # the finite element space for the displacement u_h^i

# print(f"mag_ndof={fes_mag.ndof}, disp_ndof={fes_disp.ndof},\n, dispfree_ndof={fes_disp.FreeDofs()}")
mag_gfu = GridFunction(fes_mag)
disp_gfu = GridFunction(fes_disp)
# body force and traction force

# f_body = CoefficientFunction((0,0,-1))
# g_surface = CoefficientFunction([-1 if bc=="left" else 0 for bc in mesh.GetBoundaries()])

In [5]:
# Initial conditions
mag_gfu = magfunc.give_random_magnetisation(mag_gfu)
disp_gfu = elfunc.give_random_displacement(disp_gfu)
velocity_gfu = elfunc.give_random_displacement(disp_gfu)  # An initial velocity. Should only be used once in iteration.
Draw(mag_gfu)

In [6]:
num_steps = genfunc.ceiling_division(T_MAX, K)  # This is ceiling division, using upside-down floor division.
for i in range(num_steps):
    mag_gfu = magfunc.update_magnetisation(fes_mag, mag_gfu, fes_eps_m, ALPHA, THETA, K, KAPPA)
    Redraw()
    print(f"Step {i}:\n\
          integral |m*m|dx = {Integrate(mag_gfu*mag_gfu, mesh, VOL)}")
    print(f"energy = {magfunc.magnetic_energy(mag_gfu, mesh)}")

Step 0:
          integral |m*m|dx = 0.7955463833781795
energy = 532.1262118956214
Step 1:
          integral |m*m|dx = 1.50818704708743
energy = 419.16016813446225
Step 2:
          integral |m*m|dx = 2.2508471757483353
energy = 322.86327631133844
Step 3:
          integral |m*m|dx = 2.8550603021621073
energy = 256.2999878369445
Step 4:
          integral |m*m|dx = 3.282485333193947
energy = 215.20573050211846
Step 5:
          integral |m*m|dx = 3.494730422218814
energy = 197.45631301119425
Step 6:
          integral |m*m|dx = 3.6214898389043406
energy = 188.6695511167623
Step 7:
          integral |m*m|dx = 3.7439594995654137
energy = 181.43990608199942
Step 8:
          integral |m*m|dx = 3.8492530493808683
energy = 175.18791828720208
Step 9:
          integral |m*m|dx = 3.938303187085431
energy = 169.74144339722486
Step 10:
          integral |m*m|dx = 4.069211485201512
energy = 163.12760425979218
Step 11:
          integral |m*m|dx = 4.175450985715223
energy = 155.90935502550786


KeyboardInterrupt: 

In [ ]:
# Test functions
#u = fes_disp.TrialFunction()
#psi = fes_disp.TestFunction()
# Building the linear system for the displacement
#a_disp = BilinearForm(fes_disp)
#a_disp += InnerProduct(u, psi) * dx  # <u^(i+1), ψ>
#a_disp += K*K*InnerProduct(elfunc.stress(elfunc.strain(u)), elfunc.strain(psi)) * dx  # k^2<Cε(u), ε(ψ)>

#f_disp = LinearForm(fes_disp)
#f_mag += InnerProduct(elfunc.stress(magfunc.build_strain_m(fes_eps_m, magfunc.nodal_projection(mag_gfu))), elfunc.strain(psi))* dx  # <Cε_m(Π m),ε(ψ)>
# f_mag += K*InnerProduct(diff_t u, psi)*dx  # k<d_t u^i, ψ>
#f_disp +=  InnerProduct(disp_gfu, psi) * dx  # <u^i, ψ>
# f_mag += InnerProduct(f_body, psi)*dx  # k^2 <f, ψ>
# f_mag += InnerProduct(g_surface, psi)*ds  # k^2 _/‾ g·ψ ds